# Chicago Crime Data Analysis

## Data ingestion: Dataset Setup and Loading

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# loading the data
path="../data/archive_3.zip"
df= pd.read_csv(path,low_memory=True, compression='zip')

## Preliminary Data Analysis

In [2]:
# let take a snap shot view on our data
print("============ OVERVIEW OF THE DATA =========\n")
print(f"Data info: {df.info()}\n")

print(f"Column list: {df.columns}\n")

print("\n============ Description of the data =========\n")
print(f"{df.describe(include='all')}")



============ OVERVIEW OF THE DATA =========

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7784664 entries, 0 to 7784663
Data columns (total 22 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Case Number           object 
 2   Date                  object 
 3   Block                 object 
 4   IUCR                  object 
 5   Primary Type          object 
 6   Description           object 
 7   Location Description  object 
 8   Arrest                bool   
 9   Domestic              bool   
 10  Beat                  int64  
 11  District              float64
 12  Ward                  float64
 13  Community Area        float64
 14  FBI Code              object 
 15  X Coordinate          float64
 16  Y Coordinate          float64
 17  Year                  int64  
 18  Updated On            object 
 19  Latitude              float64
 20  Longitude             float64
 21  Location              object 
dt

In [3]:
# checking first five rows
print ("\nThe last five rows")
print(df.head())



The last five rows
         ID Case Number                    Date                  Block  IUCR  \
0  10224738    HY411648  09/05/2015 01:30:00 PM        043XX S WOOD ST  0486   
1  10224739    HY411615  09/04/2015 11:30:00 AM    008XX N CENTRAL AVE  0870   
2  11646166    JC213529  09/01/2018 12:01:00 AM  082XX S INGLESIDE AVE  0810   
3  10224740    HY411595  09/05/2015 12:45:00 PM      035XX W BARRY AVE  2023   
4  10224741    HY411610  09/05/2015 01:00:00 PM    0000X N LARAMIE AVE  0560   

  Primary Type              Description Location Description  Arrest  \
0      BATTERY  DOMESTIC BATTERY SIMPLE            RESIDENCE   False   
1        THEFT           POCKET-PICKING              CTA BUS   False   
2        THEFT                OVER $500            RESIDENCE   False   
3    NARCOTICS    POSS: HEROIN(BRN/TAN)             SIDEWALK    True   
4      ASSAULT                   SIMPLE            APARTMENT   False   

   Domestic  ...  Ward  Community Area  FBI Code  X Coordinate Y C

In [4]:
# checking last five rows
print ("\nThe last five rows")
print(df.tail())


The last five rows
               ID Case Number                    Date                Block  \
7784659  12847575    JF420478  09/01/2022 05:00:00 AM      005XX W SURF ST   
7784660  12847801    JF420319  07/08/2022 12:00:00 AM  114XX S PRAIRIE AVE   
7784661  12847324    JF420102  09/27/2022 11:00:00 AM      023XX E 70TH ST   
7784662  12847570    JF420427  09/03/2022 10:25:00 AM   052XX W CARMEN AVE   
7784663  12840464    JF411839  09/26/2022 07:20:00 PM    0000X N MASON AVE   

         IUCR        Primary Type                    Description  \
7784659  2825       OTHER OFFENSE        HARASSMENT BY TELEPHONE   
7784660  1130  DECEPTIVE PRACTICE       FRAUD OR CONFIDENCE GAME   
7784661  0810               THEFT                      OVER $500   
7784662  2021           NARCOTICS         POSSESS - BARBITURATES   
7784663  143A   WEAPONS VIOLATION  UNLAWFUL POSSESSION - HANDGUN   

                    Location Description  Arrest  Domestic  ...  Ward  \
7784659                      

In [5]:
# checking the missing data
df.isna().sum()

ID                           0
Case Number                  4
Date                         0
Block                        0
IUCR                         0
Primary Type                 0
Description                  0
Location Description     10381
Arrest                       0
Domestic                     0
Beat                         0
District                    47
Ward                    614848
Community Area          613476
FBI Code                     0
X Coordinate             86848
Y Coordinate             86848
Year                         0
Updated On                   0
Latitude                 86848
Longitude                86848
Location                 86848
dtype: int64

## Data Cleaning and Preprocessing

### let change some data type first

In [6]:
# make a copy of dataset
new_df=df.copy()

## Descriptive Data Analysis(DDA)

## Visualization